## ДЗ_7 - Тема “Свёртки”

Берем отызывы за лето (из архива с материалами или предыдущего занятия)

1. Учим conv сеть для классификации
2. Рассмотреть 2-а варианта сеточек

2.1 Инициализировать tf.keras.layers.Embedding предобученными векторами взять к примеру с https://rusvectores.org/ru/

2.2 Инициализировать слой tf.keras.layers.Embedding по умолчанию (ну то есть вам ничего не делать с весами)

Сравнить две архитектуры с предобученными весами и когда tf.keras.layers.Embedding обучается сразу со всей сеточкой, что получилось лучше.

### Установим необходимые библиотеки и загрузим даные.

In [1]:
!pip install stop_words pymorphy2

In [24]:
!pip install tensorflow-addons

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 8.5 MB/s eta 0:00:00:00:010:01


In [36]:
!pip install -U gensim

In [37]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_addons as tfa

from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import keras
from keras import initializers
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
# from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import TensorBoard 
from keras.metrics import categorical_crossentropy
from keras.callbacks import EarlyStopping

import nltk
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
nltk.download("punkt")

[nltk_data] Downloading package punkt to /Users/mac/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

**Зададим гиперпараметры для обучения.**

In [7]:
max_words = 1000
max_len = 80
num_classes = 1

epochs = 20
batch_size = 512
print_batch_n = 100

**Загрузим датасет.**

In [9]:
df =  pd.read_excel('отзывы за лето.xls')
df.head(5)

,Rating,Content,Date
0,5,It just works!,2017-08-14
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14
2,5,Отлично все,2017-08-14
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14
4,5,"Очень удобно, работает быстро.",2017-08-14


**Удалим столбец Date.**

In [10]:
df.drop('Date', axis=1, inplace=True)

**Сделаем предобработку текста.**

In [16]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

**Разделим данные на test и train.**

In [11]:
df_train, df_test = train_test_split(df, test_size=0.33, random_state=42)

In [12]:
df_train.head()

,Rating,Content
3517,5,Наконец-то исправили эту чушь с неоргинальной ...
12399,5,Удобно в использовании
3833,5,Отлично
13250,5,Класс
17941,5,Удобно


In [14]:
df_test.head()

,Rating,Content
1457,5,Всё отлично!!!
19916,5,Мне нравится. Удобно. Еще ни разу не подвел!
2112,3,Иногда слишком долго приходится ждать пока зап...
15154,5,Отличное приложение. Легко отслеживать свои ра...
18329,5,Все грамотно


In [17]:
df_train['Content'] = df_train['Content'].apply(preprocess_text)
df_test['Content'] = df_test['Content'].apply(preprocess_text)

In [18]:
df_train.head()

,Rating,Content
3517,5,наконецтый исправить чушь снеоргинальный проши...
12399,5,удобно использование
3833,5,отлично
13250,5,класс
17941,5,удобно


In [19]:
df_test.head()

,Rating,Content
1457,5,отлично
19916,5,нравиться удобно разуна подвести
2112,3,приходиться ждать запуститься приложение бввае...
15154,5,отличный приложение легко отслеживать расход о...
18329,5,грамотно


**Сформируем сорварь. Проиндексируем токены.**

In [20]:
train_corpus = " ".join(df_train["Content"])
train_corpus = train_corpus.lower()

In [21]:
tokens = word_tokenize(train_corpus)
tokens_filtered = [word for word in tokens if word.isalnum()]

**Выберем наиболее часто встречающиеся токены.**

In [22]:
dist = FreqDist(tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]
tokens_filtered_top[:10]

['приложение',
 'удобно',
 'работать',
 'удобный',
 'отлично',
 'нравиться',
 'хороший',
 'отличный',
 'телефон',
 'супер']

In [23]:
vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}
len(vocabulary)

999

In [26]:
def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])
    padding = [0]*(maxlen-len(result))
    return padding + result[-maxlen:]

In [27]:
x_train = np.asarray([text_to_sequence(text, max_len) for text in df_train["Content"]], dtype=np.int32)
x_test = np.asarray([text_to_sequence(text, max_len) for text in df_test["Content"]], dtype=np.int32)

In [28]:
le = LabelEncoder()
train_enc_labels = le.fit_transform(df_train['Rating']) 
test_enc_labels = le.transform(df_test['Rating'])
le.classes_

array([1, 2, 3, 4, 5])

In [29]:
num_classes = 5
y_train = tf.keras.utils.to_categorical(train_enc_labels, num_classes=num_classes)
y_test = tf.keras.utils.to_categorical(test_enc_labels, num_classes=num_classes)

**Соберем сеть.**

In [38]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=256, input_length=max_len))
model.add(Conv1D(256, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(40))
model.add(Activation("relu"))
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 80, 256)           256000    
                                                                 
 conv1d_1 (Conv1D)           (None, 78, 256)           196864    
                                                                 
 activation_4 (Activation)   (None, 78, 256)           0         
                                                                 
 global_max_pooling1d_1 (Glo  (None, 256)              0         
 balMaxPooling1D)                                                
                                                                 
 dense_3 (Dense)             (None, 40)                10280     
                                                                 
 activation_5 (Activation)   (None, 40)                0         
                                                      

**Скомпилируем ее.**

In [39]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy', tfa.metrics.F1Score(num_classes=num_classes, average='macro')])

In [66]:
tensorboard = TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss') 

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard])
#                     callbacks=[tensorboard, early_stopping])

Epoch 1/20
25/25 [==============================] - 8s 333ms/step - loss: 0.1856 - accuracy: 0.9452 - f1_score: 0.9159 - val_loss: 1.4819 - val_accuracy: 0.7307 - val_f1_score: 0.3826
Epoch 2/20
25/25 [==============================] - 9s 350ms/step - loss: 0.1833 - accuracy: 0.9465 - f1_score: 0.9177 - val_loss: 1.5056 - val_accuracy: 0.7271 - val_f1_score: 0.3804
Epoch 3/20
25/25 [==============================] - 9s 370ms/step - loss: 0.1820 - accuracy: 0.9465 - f1_score: 0.9180 - val_loss: 1.5404 - val_accuracy: 0.7314 - val_f1_score: 0.3744
Epoch 4/20
25/25 [==============================] - 10s 400ms/step - loss: 0.1803 - accuracy: 0.9466 - f1_score: 0.9195 - val_loss: 1.5561 - val_accuracy: 0.7314 - val_f1_score: 0.3774
Epoch 5/20
25/25 [==============================] - 11s 446ms/step - loss: 0.1790 - accuracy: 0.9471 - f1_score: 0.9192 - val_loss: 1.5725 - val_accuracy: 0.7314 - val_f1_score: 0.3840
Epoch 6/20
25/25 [==============================] - 11s 437ms/step - loss: 0.1

In [67]:
score = model.evaluate(x_test, y_test, batch_size=batch_size, verbose=1)

14/14 [==============================] - 1s 91ms/step - loss: 1.6487 - accuracy: 0.7366 - f1_score: 0.3675


**Загрузим предобученную сеть.**

In [42]:
!wget http://vectors.nlpl.eu/repository/20/220.zip

--2022-11-15 16:29:40--  http://vectors.nlpl.eu/repository/20/220.zip
Распознаётся vectors.nlpl.eu (vectors.nlpl.eu)… 129.240.189.181
Подключение к vectors.nlpl.eu (vectors.nlpl.eu)|129.240.189.181|:80... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 200 OK
Длина: 638171816 (609M) [application/zip]
Сохранение в: «220.zip.1»

220.zip.1           100%[===================>] 608,61M  19,7MB/s    за 34s     

2022-11-15 16:30:13 (18,2 MB/s) - «220.zip.1» сохранён [638171816/638171816]



In [35]:
!unzip 220.zip

Archive:  220.zip
  inflating: meta.json               
  inflating: model.bin               
  inflating: model.txt               
  inflating: README                  


In [43]:
import gensim

**Загрузим векторы из предобученной модели.**

In [44]:
word_model = gensim.models.KeyedVectors.load_word2vec_format('model.bin', binary=True)

In [45]:
len(word_model)

249333

In [78]:
# embeddings_index = {}
# i = 0
# ids = []
# with open('model.txt') as f:
#     for line in f:
#       i += 1
#       word, coefs = line.split(maxsplit=1)
#       coefs = np.fromstring(coefs, "f", sep=" ")
#       if i > 1:
#         text, sem = word.split('_')
#         embeddings_index[text] = coefs
#         ids.append(text)


# print(f'Добавлено {len(embeddings_index)} векторов.')

Добавлено 205707 векторов.


In [80]:
# embedding_dim = 300
# hits = 0
# misses = 0

# embedding_matrix = np.zeros((max_words, embedding_dim))
# for word, i in vocabulary.items():
#     embedding_vector = embeddings_index.get(word)
#     if embedding_vector is not None:
#         embedding_matrix[i] = embedding_vector
#         hits += 1
#     else:
#         misses += 1

# print(f'Добавлены эмбеддинги для {hits} слов ({misses} слов не найдено).')

Добавлены эмбеддинги для 869 слов (130 слов не найдено).


**Подгоним веторы по размеру эмбеддингов из предыдущей модели.**

In [46]:
word_model_matrix = [word_model[i][:256] for i in range(1000)]

In [50]:
initializer = tf.keras.initializers.Constant(word_model_matrix)

model_w = Sequential()
model_w.add(Embedding(input_dim=max_words, output_dim=256, embeddings_initializer =initializer, input_length=max_len))
model_w.add(Conv1D(256, 3))
model_w.add(Activation("relu"))
model_w.add(GlobalMaxPool1D())
model_w.add(Dense(40))
model_w.add(Activation("relu"))
model_w.add(Dense(10))
model_w.add(Activation("relu"))
model_w.add(Dense(num_classes))
model_w.add(Activation('softmax'))
model_w.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 80, 256)           256000    
                                                                 
 conv1d_4 (Conv1D)           (None, 78, 256)           196864    
                                                                 
 activation_14 (Activation)  (None, 78, 256)           0         
                                                                 
 global_max_pooling1d_4 (Glo  (None, 256)              0         
 balMaxPooling1D)                                                
                                                                 
 dense_10 (Dense)            (None, 40)                10280     
                                                                 
 activation_15 (Activation)  (None, 40)                0         
                                                      

In [68]:
model_w.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy', tfa.metrics.F1Score(num_classes=num_classes, average='macro')])

In [69]:
tensorboard = TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss') 

history_w = model_w.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard])
#                     callbacks=[tensorboard, early_stopping])

Epoch 1/20
25/25 [==============================] - 12s 440ms/step - loss: 0.8512 - accuracy: 0.7437 - f1_score: 0.2912 - val_loss: 0.7613 - val_accuracy: 0.7300 - val_f1_score: 0.3043
Epoch 2/20
25/25 [==============================] - 11s 460ms/step - loss: 0.6012 - accuracy: 0.7913 - f1_score: 0.3244 - val_loss: 0.7926 - val_accuracy: 0.7415 - val_f1_score: 0.2851
Epoch 3/20
25/25 [==============================] - 12s 485ms/step - loss: 0.5819 - accuracy: 0.7959 - f1_score: 0.3201 - val_loss: 0.7679 - val_accuracy: 0.7422 - val_f1_score: 0.2883
Epoch 4/20
25/25 [==============================] - 13s 531ms/step - loss: 0.5746 - accuracy: 0.7981 - f1_score: 0.3270 - val_loss: 0.7649 - val_accuracy: 0.7386 - val_f1_score: 0.2876
Epoch 5/20
25/25 [==============================] - 13s 523ms/step - loss: 0.5681 - accuracy: 0.7995 - f1_score: 0.3269 - val_loss: 0.7676 - val_accuracy: 0.7430 - val_f1_score: 0.2832
Epoch 6/20
25/25 [==============================] - 12s 498ms/step - loss: 

In [70]:
score = model_w.evaluate(x_test, y_test, batch_size=batch_size, verbose=1)

14/14 [==============================] - 1s 93ms/step - loss: 0.7703 - accuracy: 0.7558 - f1_score: 0.2926


### Вывод: обе модели показали примерно одинаковые результаты по показателю accuracy - 0.7366 у модели которая обучалась сама против 0.7558 у предобученной. Однако показатель f1_score у предобученной модели существенно ниже - 0.2926 против 0.3675. Вероятно, это может быть связано с тем, что не для всех слов в нашем датасете нашлись предобученные эмбеддинги. 